In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,294 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [543 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [ ]:
# Create the vine_table. DataFrame
vinedf = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vinedf.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [ ]:
vinedf = vinedf.filter("total_votes>=20")

In [ ]:
vine_df = vinedf.filter("helpful_votes/total_votes>=0.5")
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [ ]:
vine_y_df = vine_df.filter("vine == 'Y'")
vine_y_df.show(10)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [ ]:
vine_n_df = vine_df.filter("vine == 'N'")
vine_n_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [ ]:
#I have to know, since there were none in the filtered list
total_y_df = df.filter("vine == 'Y'")
total_y_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42321535|R2NKNL4WXWIWBG|1400062217|     791531418|The Laments: A Novel|           Books|          4|            3|          3|   Y|                Y|Doh! He *almost* ...|There were times ...| 2005-09-12|
|         US|   52254603| RAIG9ZQDMBF5P|0679742034|     586032452|    Buffalo Soldiers|           Books|          5|    

In [ ]:
total_y = total_y_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
total_y.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2NKNL4WXWIWBG|          4|            3|          3|   Y|                Y|
| RAIG9ZQDMBF5P|          5|            3|          6|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [50]:
#start calculations
total_y.groupby("vine").agg({'review_id':'count'}).show()

+----+----------------+
|vine|count(review_id)|
+----+----------------+
|   Y|               2|
+----+----------------+



In [51]:
y_star_df = total_y.groupby("star_rating").agg({'review_id':'count', 'total_votes':'sum'}).withColumnRenamed('count(review_id)','reviews')
y_star_df.show()

+-----------+-------+----------------+
|star_rating|reviews|sum(total_votes)|
+-----------+-------+----------------+
|          5|      1|               6|
|          4|      1|               3|
+-----------+-------+----------------+



In [47]:
y_star_df.withColumn('percentage',y_star_df['reviews']/2).show()

+-----------+-----+----------+
|star_rating|votes|percentage|
+-----------+-----+----------+
|          5|    1|       0.5|
|          4|    1|       0.5|
+-----------+-----+----------+



In [53]:
n_star_df = vine_n_df.groupby("star_rating").agg({'review_id':'count', 'total_votes':'sum'}).withColumnRenamed('count(review_id)','reviews')
n_star_df.show()

+-----------+-------+----------------+
|star_rating|reviews|sum(total_votes)|
+-----------+-------+----------------+
|          1|  42547|         2115960|
|          3|  31387|         1470672|
|          5| 242889|        11703999|
|          4|  60898|         2705705|
|          2|  26086|         1219010|
+-----------+-------+----------------+



In [39]:
vine_n_df.groupby("vine").agg({'total_votes':'sum', 'review_id':'count'}).show()

+----+----------------+----------------+
|vine|count(review_id)|sum(total_votes)|
+----+----------------+----------------+
|   N|          403807|        19215346|
+----+----------------+----------------+



In [48]:
#It is late enough that I would rather copy the total number of votes to divide by than take the extra time to work out how to have the program find and put in there
n_star_df.withColumn('percentage',n_star_df['reviews']/ 403807).show()

+-----------+-------+-------------------+
|star_rating|reviews|         percentage|
+-----------+-------+-------------------+
|          1|  42547| 0.1053646915481901|
|          3|  31387|0.07772772636432751|
|          5| 242889| 0.6014977452099641|
|          4|  60898|0.15080966897552545|
|          2|  26086|0.06460016790199279|
+-----------+-------+-------------------+

